In [9]:
import torch
import librosa
from transformers import WhisperProcessor, WhisperForConditionalGeneration

# 1. Whisper Tiny 모델 로드
model_name = "openai/whisper-tiny"
model = WhisperForConditionalGeneration.from_pretrained(model_name)
processor = WhisperProcessor.from_pretrained(model_name)

# 2. 모델을 평가 모드로 전환
model.eval()

# 3. 오디오 파일 로드 및 처리
def load_audio(audio_path, sampling_rate=16000):
    audio, sr = librosa.load(audio_path, sr=sampling_rate)  # 16kHz로 샘플링
    return audio

audio_path = "data1.wav"  # 음원 파일 경로
audio = load_audio(audio_path)

# 4. 입력 데이터 준비
input_features = processor(audio, sampling_rate=16000, return_tensors="pt").input_features

# 5. 모델 추론
with torch.no_grad():
    predicted_ids = model.generate(input_features)

# 6. 결과 디코딩
transcription = processor.batch_decode(predicted_ids, skip_special_tokens=True)[0]

# 7. 결과 출력
print("Transcription:", transcription)


Due to a bug fix in https://github.com/huggingface/transformers/pull/28687 transcription using a multilingual Whisper will default to language detection followed by transcription instead of translation to English.This might be a breaking change for your use case. If you want to instead always translate your audio to English, make sure to pass `language='en'`.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Transcription:  오늘 여자 올려 구경하냐? 뭐 보고 있냐 지금? 지금 이 여자가 보고 라코랑 매종 기치내에서 그... 뭐야, 카라티 하나씩 커플로 사고. 얼마 안 돼? 라코를 두 개에서 15만원 샀고 매종을 두 개에서 20,000만원. 괜찮네 두 개에 2,000이면. 그래서 이렇게 내게 사서 40개 안 들었으니까.


In [15]:
import torch
import librosa
from transformers import WhisperProcessor, WhisperForConditionalGeneration

# 1. Whisper Tiny 모델 로드
model_name = "openai/whisper-tiny"
model = WhisperForConditionalGeneration.from_pretrained(model_name)
processor = WhisperProcessor.from_pretrained(model_name)

# 2. 모델을 평가 모드로 전환
model.eval()

# 3. 1.5초 단위 음성 인식 함수
def fast_transcription(audio_path, chunk_size=1.5, sampling_rate=16000):
    """
    1.5초 단위로 오디오 데이터를 처리하여 빠른 텍스트 출력
    - chunk_size: 처리할 오디오 길이 (초 단위, 기본값 1.5초)
    """
    # 오디오 파일 로드
    audio, sr = librosa.load(audio_path, sr=sampling_rate)
    
    # 1.5초 단위 처리 (chunk_size 초 길이로 나눔)
    chunk_length = int(chunk_size * sampling_rate)  # 1.5초에 해당하는 샘플 수
    total_chunks = len(audio) // chunk_length  # 총 처리할 청크 수

    for i in range(total_chunks):
        # 현재 청크 추출
        start = i * chunk_length
        end = start + chunk_length
        audio_chunk = audio[start:end]
        
        # 입력 데이터 준비
        input_features = processor(audio_chunk, sampling_rate=sampling_rate, return_tensors="pt").input_features
        
        # 모델 추론
        with torch.no_grad():
            predicted_ids = model.generate(input_features)
        
        # 결과 디코딩
        transcription = processor.batch_decode(predicted_ids, skip_special_tokens=True)[0]
        
        # 결과 출력
        print(f"[{start / sampling_rate:.2f}s - {end / sampling_rate:.2f}s]: {transcription}")

# 4. 실행
audio_path = "data1.wav"  # 음원 파일 경로
fast_transcription(audio_path, chunk_size=1.5, sampling_rate=16000)


[0.00s - 1.50s]:  Oh
[1.50s - 3.00s]:  오늘 여자 올려 부검하냐?
[3.00s - 4.50s]:  Oh
[4.50s - 6.00s]: 哇,我不过应该这个
[6.00s - 7.50s]:  시간은
[7.50s - 9.00s]:  그리고 이곳에서
[9.00s - 10.50s]:  랩소라운 매주
[10.50s - 12.00s]:  좋은 기침나에서
[12.00s - 13.50s]:  Mmm, good.
[13.50s - 15.00s]: カラッキー話し
[15.00s - 16.50s]:  어마어마한 택시
[16.50s - 18.00s]:  لقوم
[18.00s - 19.50s]:  두 개에서 15만원 샀고
[19.50s - 21.00s]:  음
[21.00s - 22.50s]:  두 개에서 20천만 원
[22.50s - 24.00s]:  괜찮네.
[24.00s - 25.50s]:  on.
[25.50s - 27.00s]:  Excuse me. Okay.
[27.00s - 28.50s]:  4개 사서 40위 안 들었습니다.
[28.50s - 30.00s]: うん
[30.00s - 31.50s]:  라인업치 공개는 잔치
[31.50s - 33.00s]:  안녕, 압시고.
[33.00s - 34.50s]:  또 뭐 브랜드 치고는
[34.50s - 36.00s]:  사기 편의
[36.00s - 37.50s]:  비싸긴 한데
[37.50s - 39.00s]:  If you think of her love
[39.00s - 40.50s]: わしにか
[40.50s - 42.00s]:  밥은 너무 올려
[42.00s - 43.50s]:  다 펜정해서 먹었어.
[43.50s - 45.00s]:  거기에 참 살려
[45.00s - 46.50s]:  맛있는데 많은데
[46.50s - 48.00s]:  너무 멀어.
[48.00s - 49.50s]:  Ağakıla.
[49.50s - 51.00s]:  갔다만 시작해. 다시 올라오면
[51.00s - 52.50s]:

### 한국어만인식 설정

In [17]:
import torch
import librosa
from transformers import WhisperProcessor, WhisperForConditionalGeneration

# 모델 로드
model_name = "openai/whisper-tiny"
model = WhisperForConditionalGeneration.from_pretrained(model_name)
processor = WhisperProcessor.from_pretrained(model_name)

# 모델 평가 모드
model.eval()

# 1.5초 단위 음성 인식 함수 (한국어만 인식)
def fast_transcription(audio_path, chunk_size=1.5, sampling_rate=16000):
    """
    1.5초 단위로 오디오 데이터를 처리하여 빠른 텍스트 출력
    - chunk_size: 처리할 오디오 길이 (초 단위, 기본값 1.5초)
    - 한국어만 인식되도록 강제 설정
    """
    # 오디오 파일 로드
    audio, sr = librosa.load(audio_path, sr=sampling_rate)
    
    # 1.5초 단위 처리
    chunk_length = int(chunk_size * sampling_rate)  # 1.5초에 해당하는 샘플 수
    total_chunks = len(audio) // chunk_length  # 총 처리할 청크 수

    for i in range(total_chunks):
        # 현재 청크 추출
        start = i * chunk_length
        end = start + chunk_length
        audio_chunk = audio[start:end]
        
        # 입력 데이터 준비
        input_features = processor(audio_chunk, sampling_rate=sampling_rate, return_tensors="pt").input_features
        
        # 모델 추론 (한국어 강제 설정)
        with torch.no_grad():
            predicted_ids = model.generate(
                input_features,
                forced_decoder_ids=processor.get_decoder_prompt_ids(language="ko", task="transcribe")
            )
        
        # 결과 디코딩
        transcription = processor.batch_decode(predicted_ids, skip_special_tokens=True)[0]
        
        # 결과 출력
        print(f"[{start / sampling_rate:.2f}s - {end / sampling_rate:.2f}s]: {transcription}")

# 실행
audio_path = "data1.wav"  # 음원 파일 경로
fast_transcription(audio_path, chunk_size=1.5, sampling_rate=16000)


[0.00s - 1.50s]:  아
[1.50s - 3.00s]:  오늘 여자 올려 부검하냐?
[3.00s - 4.50s]:  어
[4.50s - 6.00s]:  와우 보고 있냐?
[6.00s - 7.50s]:  시간은
[7.50s - 9.00s]:  그리고 이곳에서
[9.00s - 10.50s]:  랩소라운 매주
[10.50s - 12.00s]:  좋은 기침나에서
[12.00s - 13.50s]:  음...
[13.50s - 15.00s]:  가락이 안 하시
[15.00s - 16.50s]:  어마어마한 택시
[16.50s - 18.00s]:  랩호
[18.00s - 19.50s]:  두 개에서 15만원 샀고
[19.50s - 21.00s]:  음
[21.00s - 22.50s]:  두 개에서 20천만 원
[22.50s - 24.00s]:  괜찮네.
[24.00s - 25.50s]:  한 번 뿌려줬어
[25.50s - 27.00s]:  오케이
[27.00s - 28.50s]:  4개 사서 40위 안 들었습니다.
[28.50s - 30.00s]:  좋으니까
[30.00s - 31.50s]:  라인업치 공개는 잔치
[31.50s - 33.00s]:  안녕, 압시고.
[33.00s - 34.50s]:  또 뭐 브랜드 치고는
[34.50s - 36.00s]:  사기 편의
[36.00s - 37.50s]:  비싸긴 한데
[37.50s - 39.00s]:  그런데 비싹을 하려고
[39.00s - 40.50s]:  아, 아, 아, 아, 아, 아, 아, 아, 아, 아, 아, 아, 아, 아, 아, 아, 아, 아, 아, 아, 아, 아, 아, 아, 아, 아, 아, 아, 아, 아, 아, 아, 아, 아, 아, 아, 아, 아, 아, 아, 아, 아, 아, 아, 아, 아, 아, 아, 아, 아, 아, 아, 아, 아, 아, 아, 아, 아, 아, 아, 아, 아, 아, 아, 아, 아, 아, 아, 아, 아, 아, 아, 아, 아, 아 아, 아 아, 아 아, 아 아, 아 아, 아 아, 아 아, 아 

In [31]:
import torch
import librosa
from transformers import WhisperProcessor, WhisperForConditionalGeneration

# 모델 로드
model_name = "openai/whisper-tiny"
model = WhisperForConditionalGeneration.from_pretrained(model_name)
processor = WhisperProcessor.from_pretrained(model_name)

# 모델 평가 모드
model.eval()

# 2초 단위 음성 인식 함수 (한국어만 인식)
def fast_transcription(audio_path, chunk_size=2.0, sampling_rate=16000):
    """
    2초 단위로 오디오 데이터를 처리하여 빠른 텍스트 출력
    - chunk_size: 처리할 오디오 길이 (초 단위, 기본값 2초)
    - 한국어만 인식되도록 강제 설정
    """
    # 오디오 파일 로드
    audio, sr = librosa.load(audio_path, sr=sampling_rate)
    
    # 2초 단위 처리
    chunk_length = int(chunk_size * sampling_rate)  # 2초에 해당하는 샘플 수
    total_chunks = len(audio) // chunk_length  # 총 처리할 청크 수

    for i in range(total_chunks):
        # 현재 청크 추출
        start = i * chunk_length
        end = start + chunk_length
        audio_chunk = audio[start:end]
        
        # 입력 데이터 준비
        input_features = processor(audio_chunk, sampling_rate=sampling_rate, return_tensors="pt").input_features
        
        # 모델 추론 (한국어 강제 설정)
        with torch.no_grad():
            predicted_ids = model.generate(
                input_features,
                forced_decoder_ids=processor.get_decoder_prompt_ids(language="ko", task="transcribe")
            )
        
        # 결과 디코딩
        transcription = processor.batch_decode(predicted_ids, skip_special_tokens=True)[0]
        
        # 결과 출력
        print(f"[{start / sampling_rate:.2f}s - {end / sampling_rate:.2f}s]: {transcription}")

# 실행
audio_path = "data1.wav"  # 음원 파일 경로
fast_transcription(audio_path, chunk_size=2.0, sampling_rate=16000)


[0.00s - 2.00s]:  너네 여자
[2.00s - 4.00s]:  아, 아, 아, 아, 아, 아, 아, 아, 아, 아, 아, 아, 아, 아, 아, 아, 아, 아, 아, 아, 아, 아, 아, 아, 아, 아, 아, 아, 아, 아, 아, 아, 아, 아, 아, 아, 아, 아, 아, 아, 아, 아, 아, 아, 아, 아 아, 아 아, 아 아, 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아
[4.00s - 6.00s]:  와우, 뭐고 있냐?
[6.00s - 8.00s]:  시간은?
[8.00s - 10.00s]:  뭐 이곳적으로 보고
[10.00s - 12.00s]:  매종 기칭해서
[12.00s 

In [39]:
import torch
import librosa
import numpy as np
from transformers import WhisperProcessor, WhisperForConditionalGeneration

# 모델 로드
model_name = "openai/whisper-tiny"
model = WhisperForConditionalGeneration.from_pretrained(model_name)
processor = WhisperProcessor.from_pretrained(model_name)

# 모델 평가 모드
model.eval()

# 무음인지 판단하는 함수
def is_silent(audio_chunk, threshold=0.01):
    """
    오디오 청크가 무음인지 확인
    - threshold: 무음으로 간주할 최대 진폭 값 (기본값 0.01)
    """
    return np.max(np.abs(audio_chunk)) < threshold

# 2초 단위 음성 인식 함수 (한국어만 인식)
def fast_transcription(audio_path, chunk_size=2.0, sampling_rate=16000):
    """
    2초 단위로 오디오 데이터를 처리하여 빠른 텍스트 출력
    - chunk_size: 처리할 오디오 길이 (초 단위, 기본값 2초)
    - 한국어만 인식되도록 강제 설정
    """
    # 오디오 파일 로드
    audio, sr = librosa.load(audio_path, sr=sampling_rate)
    
    # 2초 단위 처리
    chunk_length = int(chunk_size * sampling_rate)  # 2초에 해당하는 샘플 수
    total_chunks = len(audio) // chunk_length  # 총 처리할 청크 수

    for i in range(total_chunks):
        # 현재 청크 추출
        start = i * chunk_length
        end = start + chunk_length
        audio_chunk = audio[start:end]
        
        # 무음 청크 건너뛰기
        if is_silent(audio_chunk):
            continue
        
        # 입력 데이터 준비
        input_features = processor(audio_chunk, sampling_rate=sampling_rate, return_tensors="pt").input_features
        
        # 모델 추론 (한국어 강제 설정)
        with torch.no_grad():
            predicted_ids = model.generate(
                input_features,
                forced_decoder_ids=processor.get_decoder_prompt_ids(language="ko", task="transcribe")
            )
        
        # 결과 디코딩
        transcription = processor.batch_decode(predicted_ids, skip_special_tokens=True)[0]
        
        # 결과 출력
        print(f"[{start / sampling_rate:.2f}s - {end / sampling_rate:.2f}s]: {transcription}")

# 실행
audio_path = "data1.wav"  # 음원 파일 경로
fast_transcription(audio_path, chunk_size=2.0, sampling_rate=16000)


[0.00s - 2.00s]:  너네 여자
[2.00s - 4.00s]:  아, 아, 아, 아, 아, 아, 아, 아, 아, 아, 아, 아, 아, 아, 아, 아, 아, 아, 아, 아, 아, 아, 아, 아, 아, 아, 아, 아, 아, 아, 아, 아, 아, 아, 아, 아, 아, 아, 아, 아, 아, 아, 아, 아, 아, 아 아, 아 아, 아 아, 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아
[4.00s - 6.00s]:  와우, 뭐고 있냐?
[6.00s - 8.00s]:  시간은?
[8.00s - 10.00s]:  뭐 이곳적으로 보고
[10.00s - 12.00s]:  매종 기칭해서
[12.00s 

In [49]:
import torch
import librosa
import numpy as np
from transformers import WhisperProcessor, WhisperForConditionalGeneration

# 모델 로드
model_name = "openai/whisper-small"
model = WhisperForConditionalGeneration.from_pretrained(model_name)
processor = WhisperProcessor.from_pretrained(model_name)

# 모델 평가 모드
model.eval()

# 무음인지 판단하는 함수
def is_silent(audio_chunk, threshold=0.01):
    """
    오디오 청크가 무음인지 확인
    - threshold: 무음으로 간주할 최대 진폭 값 (기본값 0.01)
    """
    return np.max(np.abs(audio_chunk)) < threshold

# 2초 단위 음성 인식 함수 (한국어만 인식)
def fast_transcription(audio_path, chunk_size=2.0, sampling_rate=16000):
    """
    2초 단위로 오디오 데이터를 처리하여 빠른 텍스트 출력
    - chunk_size: 처리할 오디오 길이 (초 단위, 기본값 2초)
    - 한국어만 인식되도록 강제 설정
    """
    # 오디오 파일 로드
    audio, sr = librosa.load(audio_path, sr=sampling_rate)
    
    # 2초 단위 처리
    chunk_length = int(chunk_size * sampling_rate)  # 2초에 해당하는 샘플 수
    total_chunks = len(audio) // chunk_length  # 총 처리할 청크 수

    for i in range(total_chunks):
        # 현재 청크 추출
        start = i * chunk_length
        end = start + chunk_length
        audio_chunk = audio[start:end]
        
        # 무음 청크 건너뛰기
        if is_silent(audio_chunk):
            continue
        
        # 입력 데이터 준비
        input_features = processor(audio_chunk, sampling_rate=sampling_rate, return_tensors="pt").input_features
        
        # 모델 추론 (한국어 강제 설정)
        with torch.no_grad():
            predicted_ids = model.generate(
                input_features,
                forced_decoder_ids=processor.get_decoder_prompt_ids(language="ko", task="transcribe")
            )
        
        # 결과 디코딩
        transcription = processor.batch_decode(predicted_ids, skip_special_tokens=True)[0]
        
        # 결과 출력
        print(f"[{start / sampling_rate:.2f}s - {end / sampling_rate:.2f}s]: {transcription}")

# 실행
audio_path = "data1.wav"  # 음원 파일 경로
fast_transcription(audio_path, chunk_size=2.0, sampling_rate=16000)


config.json:   0%|          | 0.00/1.97k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/967M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/3.87k [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/185k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/283k [00:00<?, ?B/s]

...

merges.txt:   0%|          | 0.00/494k [00:00<?, ?B/s]

normalizer.json:   0%|          | 0.00/52.7k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/34.6k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.19k [00:00<?, ?B/s]

[0.00s - 2.00s]:  어
[2.00s - 4.00s]:  나 원래 구경하냐? 어
[4.00s - 6.00s]:  뭐하고 뭐 보고 있냐 지금
[6.00s - 8.00s]:  지금은? 지금.
[8.00s - 10.00s]:  뭐 이것저것 보고
[10.00s - 12.00s]:  맵고랑 매종키친에서
[12.00s - 14.00s]:  그 뭐야 카라
[14.00s - 16.00s]:  어떻게 하나씩 커플로 사고
[16.00s - 18.00s]:  락쿠
[18.00s - 20.00s]:  2개에서 15만원 샀고
[20.00s - 22.00s]:  매종 2개에서 23만원
[22.00s - 24.00s]:  괜찮네 2개는 2,3이면
[24.00s - 26.00s]:  아, 그래서...
[26.00s - 28.00s]:  이렇게 총 4개 사서 40...
[28.00s - 30.00s]:  이 안 들었으니까
[30.00s - 32.00s]:  라인업 치고 괜찮지? 그런데 라인업 치고
[32.00s - 34.00s]:  브랜드..뭐..
[34.00s - 36.00s]:  근데 치고는 자기 맞네
[36.00s - 38.00s]:  비싸긴 한데
[38.00s - 40.00s]:  비슷한거 하고 왔으니까
[40.00s - 42.00s]:  그래 알았어 밥은 뭐 먹었냐
[42.00s - 44.00s]:  다 편의점 가서 먹었어
[44.00s - 46.00s]:  거기 2000살 맛있나봐
[46.00s - 48.00s]:  거기까지 너무 멀어 삼슨 가야 돼
[48.00s - 50.00s]:  아 그래?
[50.00s - 52.00s]:  다시 올라오면
[52.00s - 54.00s]:  그래서
[54.00s - 56.00s]:  진짜 잘했어
[56.00s - 58.00s]:  음식값이 얼마나 비쌌는데?
[58.00s - 60.00s]:  여기서 먹으면 인낙은 만발청식 나가더라고
[60.00s - 62.00s]:  아 그 정도는 하지 서울이니까
[62.00s - 64.00s]:

In [40]:
import torch
import librosa
import numpy as np
from transformers import WhisperProcessor, WhisperForConditionalGeneration

# 모델 로드
model_name = "openai/whisper-tiny"
model = WhisperForConditionalGeneration.from_pretrained(model_name)
processor = WhisperProcessor.from_pretrained(model_name)

# 모델 평가 모드
model.eval()

# 무음인지 판단하는 함수
def is_silent(audio_chunk, threshold=0.01):
    """
    오디오 청크가 무음인지 확인
    - threshold: 무음으로 간주할 최대 진폭 값 (기본값 0.01)
    """
    return np.max(np.abs(audio_chunk)) < threshold

# 3초 단위 음성 인식 함수 (한국어만 인식)
def fast_transcription(audio_path, chunk_size=3.0, sampling_rate=16000):
    """
    3초 단위로 오디오 데이터를 처리하여 빠른 텍스트 출력
    - chunk_size: 처리할 오디오 길이 (초 단위, 기본값 3초)
    - 한국어만 인식되도록 강제 설정
    """
    # 오디오 파일 로드
    audio, sr = librosa.load(audio_path, sr=sampling_rate)
    
    # 3초 단위 처리
    chunk_length = int(chunk_size * sampling_rate)  # 3초에 해당하는 샘플 수
    total_chunks = len(audio) // chunk_length  # 총 처리할 청크 수

    for i in range(total_chunks):
        # 현재 청크 추출
        start = i * chunk_length
        end = start + chunk_length
        audio_chunk = audio[start:end]
        
        # 무음 청크 건너뛰기
        if is_silent(audio_chunk):
            continue
        
        # 입력 데이터 준비
        input_features = processor(audio_chunk, sampling_rate=sampling_rate, return_tensors="pt").input_features
        
        # 모델 추론 (한국어 강제 설정)
        with torch.no_grad():
            predicted_ids = model.generate(
                input_features,
                forced_decoder_ids=processor.get_decoder_prompt_ids(language="ko", task="transcribe")
            )
        
        # 결과 디코딩
        transcription = processor.batch_decode(predicted_ids, skip_special_tokens=True)[0]
        
        # 결과 출력
        print(f"[{start / sampling_rate:.2f}s - {end / sampling_rate:.2f}s]: {transcription}")

# 실행
audio_path = "data1.wav"  # 음원 파일 경로
fast_transcription(audio_path, chunk_size=3.0, sampling_rate=16000)

[0.00s - 3.00s]:  너네 여자 원래 부검하냐?
[3.00s - 6.00s]:  와우, 뭐고 있냐?
[6.00s - 9.00s]:  지금 뭐 이곳적으로 보고
[9.00s - 12.00s]:  라토랑 매종 기칭에서
[12.00s - 15.00s]:  그...
[15.00s - 18.00s]:  엄만드
[18.00s - 21.00s]:  2개에서 15만원 샀고
[21.00s - 24.00s]:  2개에서 2000만 원.
[24.00s - 27.00s]:  아, 프리스 오, 그 is, 이렇게
[27.00s - 30.00s]:  4개에서 40개 안 들었으니까.
[30.00s - 33.00s]:  라인 없이 공개는 괜찮지? 그런 데 라인 없이 공개
[33.00s - 36.00s]:  뭐 브랜드 치고는 자기 다네
[36.00s - 39.00s]:  비싸긴 한데 비싸고 하려고
[39.00s - 42.00s]:  아우 아우 아우 아우 아우 아우 아우 아우 아우 아우 아우 아우 아우 아우 아우 아우 아우 아우 아우 아우 아우 아우 아우 아우 아우 아우 아우 아우 아우 아우 아우 아우 아우 아우 아우 아우 아우 아우 아우 아우 아우 아우 아우 아우 아우 아우 아우 아우 아우 아우 아우 아우 아우 아우 아우 아우 아우 아우 아우 아우 아우 아우 아우 아우 아우 아우 아우 아우 아우 아우 아우 아우 아우 아우 아우 아우 아우 아우 아우 아우 아우 아우 아우 아우 아우 아우 아우 아우 아우 아우 아우 아우 아우 아우 아우 아우 아우 아우 아우 아우 아우 아우 아우 아우 아우 아우 아우 아우 아우 아우 아우 아우 아우 아우 아우 아우 아우 아우 아우 아우 아우 아우 아우 아우 아우 아우 아우 아우 아우 아우 아우 아우 아우 아우 아우 아우 아우 아우 아우 아우 아우 아우 아우 아우 아우 아우 아우 아우 아우 아우 아우 아우 아우 아우 아우 아우 아우 아우 아우 아우 아우 아우 아우 아우 아우 아우 아우 아우 아우 아우 아우 아우 아우 아우 아우 아우 아우 아우 아우 아우 아

In [42]:
import torch
import librosa
import numpy as np
from transformers import WhisperProcessor, WhisperForConditionalGeneration

# 모델 로드
model_name = "openai/whisper-tiny"
model = WhisperForConditionalGeneration.from_pretrained(model_name)
processor = WhisperProcessor.from_pretrained(model_name)

# 모델 평가 모드
model.eval()

# 무음인지 판단하는 함수
def is_silent(audio_chunk, threshold=0.01):
    """
    오디오 청크가 무음인지 확인
    - threshold: 무음으로 간주할 최대 진폭 값 (기본값 0.01)
    """
    return np.max(np.abs(audio_chunk)) < threshold

# 4초 단위 음성 인식 함수 (한국어만 인식)
def fast_transcription(audio_path, chunk_size=4.0, sampling_rate=16000):
    """
    4초 단위로 오디오 데이터를 처리하여 빠른 텍스트 출력
    - chunk_size: 처리할 오디오 길이 (초 단위, 기본값 4초)
    - 한국어만 인식되도록 강제 설정
    """
    # 오디오 파일 로드
    audio, sr = librosa.load(audio_path, sr=sampling_rate)
    
    # 4초 단위 처리
    chunk_length = int(chunk_size * sampling_rate)  # 4초에 해당하는 샘플 수
    total_chunks = len(audio) // chunk_length  # 총 처리할 청크 수

    for i in range(total_chunks + 1):  # 마지막 청크까지 포함
        # 현재 청크 추출
        start = i * chunk_length
        end = start + chunk_length
        
        # 마지막 청크 처리 (길이가 chunk_length보다 짧을 수 있음)
        if start >= len(audio):
            break
        if end > len(audio):
            end = len(audio)
        
        audio_chunk = audio[start:end]
        
        # 무음 청크 건너뛰기
        if is_silent(audio_chunk):
            continue
        
        # 입력 데이터 준비
        input_features = processor(audio_chunk, sampling_rate=sampling_rate, return_tensors="pt").input_features
        
        # 모델 추론 (한국어 강제 설정)
        with torch.no_grad():
            predicted_ids = model.generate(
                input_features,
                forced_decoder_ids=processor.get_decoder_prompt_ids(language="ko", task="transcribe")
            )
        
        # 결과 디코딩
        transcription = processor.batch_decode(predicted_ids, skip_special_tokens=True)[0]
        
        # 결과 출력
        print(f"[{start / sampling_rate:.2f}s - {end / sampling_rate:.2f}s]: {transcription}")

# 실행
audio_path = "data1.wav"  # 음원 파일 경로
fast_transcription(audio_path, chunk_size=4.0, sampling_rate=16000)


[0.00s - 4.00s]:  너네 여자 원래 부검하냐?
[4.00s - 8.00s]:  와우 보고 있냐 지금은
[8.00s - 12.00s]:  뭐 이것저것 보고 라투랑 매종 기칭해서
[12.00s - 16.00s]:  그 카라티 하나씩 커플로 사고
[16.00s - 20.00s]:  랩호우는 두 개에서 15만원 샀고
[20.00s - 24.00s]:  매종어 두 개에서 20천만 원.
[24.00s - 28.00s]:  그래서 이렇게 형 4개 사서 40
[28.00s - 32.00s]:  이 안 들었으니까 라인업치고 괜찮지? 그런 데 라인업치고
[32.00s - 36.00s]:  브랜드 치고는 자기 다네
[36.00s - 40.00s]:  좀 비싸긴 한데, 비싸고 하러 왔으니까.
[40.00s - 44.00s]:  그래 알았어 밥은 먹느냐? 다 편집어서 먹었어
[44.00s - 48.00s]:  거기에 참 살고 맛있는데 많은데? 거기에 너무 멀어 삼아간 거야
[48.00s - 52.00s]:  아 그래? 응. 갔다만씩 해야 돼. 이렇게 다시 올라오면.
[52.00s - 56.00s]:  그래서 좀 비찾았어?
[56.00s - 60.00s]:  여기에서 먹으면 맘발청신 나가도라고
[60.00s - 64.00s]:  그 정도는 사실 서울이니까
[64.00s - 68.00s]:  야 무슨 신만 어차피 웃으면서 멈탈 천 원에
[68.00s - 72.00s]:  밥은 심망자를 예상한다고
[72.00s - 76.00s]:  심학은 자연이 20만원에 의사를 하고 왔는데 가면 많아 쳐다리 이내로 생각했을니
[76.00s - 80.00s]:  아유 원래 그 정도 하지는 무슨 메뉴가 안 발전하는 정도
[80.00s - 84.00s]:  무슨 물냉량을 많이 참쳐서 이런데?
[84.00s - 87.54s]:  알았어 재미있게 놀아


In [1]:
import torch
import librosa
import numpy as np
from transformers import WhisperProcessor, WhisperForConditionalGeneration

# 모델 로드
model_name = "openai/whisper-small"
model = WhisperForConditionalGeneration.from_pretrained(model_name)
processor = WhisperProcessor.from_pretrained(model_name)

# 모델 평가 모드
model.eval()

# 무음인지 판단하는 함수
def is_silent(audio_chunk, threshold=0.01):
    """
    오디오 청크가 무음인지 확인
    - threshold: 무음으로 간주할 최대 진폭 값 (기본값 0.01)
    """
    return np.max(np.abs(audio_chunk)) < threshold

# 4초 단위 음성 인식 함수 (한국어만 인식)
def fast_transcription(audio_path, chunk_size=7.0, sampling_rate=16000):
    """
    4초 단위로 오디오 데이터를 처리하여 빠른 텍스트 출력
    - chunk_size: 처리할 오디오 길이 (초 단위, 기본값 4초)
    - 한국어만 인식되도록 강제 설정
    """
    # 오디오 파일 로드
    audio, sr = librosa.load(audio_path, sr=sampling_rate)
    
    # 4초 단위 처리
    chunk_length = int(chunk_size * sampling_rate)  # 4초에 해당하는 샘플 수
    total_chunks = len(audio) // chunk_length  # 총 처리할 청크 수

    for i in range(total_chunks + 1):  # 마지막 청크까지 포함
        # 현재 청크 추출
        start = i * chunk_length
        end = start + chunk_length
        
        # 마지막 청크 처리 (길이가 chunk_length보다 짧을 수 있음)
        if start >= len(audio):
            break
        if end > len(audio):
            end = len(audio)
        
        audio_chunk = audio[start:end]
        
        # 무음 청크 건너뛰기
        if is_silent(audio_chunk):
            continue
        
        # 입력 데이터 준비
        input_features = processor(audio_chunk, sampling_rate=sampling_rate, return_tensors="pt").input_features
        
        # 모델 추론 (한국어 강제 설정)
        with torch.no_grad():
            predicted_ids = model.generate(
                input_features,
                forced_decoder_ids=processor.get_decoder_prompt_ids(language="ko", task="transcribe")
            )
        
        # 결과 디코딩
        transcription = processor.batch_decode(predicted_ids, skip_special_tokens=True)[0]
        
        # 결과 출력
        print(f"[{start / sampling_rate:.2f}s - {end / sampling_rate:.2f}s]: {transcription}")

# 실행
audio_path = "data1.wav"  # 음원 파일 경로
fast_transcription(audio_path, chunk_size=4.0, sampling_rate=16000)


The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


[0.00s - 4.00s]:  너네 여주아울래 구경하냐?
[4.00s - 8.00s]:  뭐하고 뭐 보고 있냐 지금은? 지금
[8.00s - 12.00s]:  뭐 이것저것 보고 라코랑 매종키친에서
[12.00s - 16.00s]:  그 뭐야 카라티 하나씩 커플로 사고 얼마
[16.00s - 20.00s]:  락코는 2개에서 15만원 샀고
[20.00s - 24.00s]:  매종은 2개에서 23만원
[24.00s - 28.00s]:  그래서 총 4개 사서 40
[28.00s - 32.00s]:  이 안 들었으니까 라인업 치고 괜찮지? 그런데 라인업 치고
[32.00s - 36.00s]:  브랜드 치고는 자기 맞네
[36.00s - 40.00s]:  비싸긴 한데 비싸고 가고 왔으니까
[40.00s - 44.00s]:  그래 알았어 밥은 뭐 먹었냐? 밥 편집만 가서 먹었어
[44.00s - 48.00s]:  거기 2000살 밥 맛있는 데 많은데 거기까지 너무 멀어 30분 가야 돼
[48.00s - 52.00s]:  아 그래? 응. 갔다 오면 1시간이 됐다. 다시 올라가.
[52.00s - 56.00s]:  그래서 좀 비찼었어
[56.00s - 60.00s]:  그냥 거기 음식값이 얼마나 비쌌는데? 여기서 먹으면 인낙한 만발청식 나가더라고
[60.00s - 64.00s]:  아 그 정도는 하지 서울이니까
[64.00s - 68.00s]:  야 무슨.. 10만원짜리 옷 사면서 18000원에
[68.00s - 72.00s]:  놀래 웃겨 죽겠네
[72.00s - 76.00s]:  10만원 정도 예상을 하고 왔는데 답은 15,000원 이내로 생각했으니까
[76.00s - 80.00s]:  그 정도 하지 무슨 메뉴가 안 8000원 정도야
[80.00s - 84.00s]:  했는데 무슨 물 냉면 한에 13,000원이던데?
[84.00s - 87.54s]:  알았어 재밌게 놀아


### ko speech model

In [27]:
import zipfile
import os

zip_path = "kospeech-latest.zip"
extract_path = "kospeech-latest"

# 압축 풀기
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print(f"압축이 풀렸습니다: {extract_path}")

압축이 풀렸습니다: kospeech-latest


In [45]:
import torch
import librosa
from kospeech.kospeech import KoSpeechModel

ImportError: cannot import name 'KoSpeechModel' from 'kospeech.kospeech' (unknown location)

In [ ]:
# 모델 로드
model = KoSpeechModel("kospeech-latest")

# 모델 평가 모드
model.eval()

# 2초 단위 음성 인식 함수 (한국어만 인식)
def fast_transcription(audio_path, chunk_size=2.0, sampling_rate=16000):
    """
    2초 단위로 오디오 데이터를 처리하여 빠른 텍스트 출력
    - chunk_size: 처리할 오디오 길이 (초 단위, 기본값 2초)
    - 한국어만 인식되도록 강제 설정
    """
    # 오디오 파일 로드
    audio, sr = librosa.load(audio_path, sr=sampling_rate)
    
    # 2초 단위 처리
    chunk_length = int(chunk_size * sampling_rate)  # 2초에 해당하는 샘플 수
    total_chunks = len(audio) // chunk_length  # 총 처리할 청크 수

    for i in range(total_chunks):
        # 현재 청크 추출
        start = i * chunk_length
        end = start + chunk_length
        audio_chunk = audio[start:end]
        
        # 입력 데이터 준비
        input_features = model.processor(audio_chunk, sampling_rate=sampling_rate, return_tensors="pt").input_features
        
        # 모델 추론 (한국어 강제 설정)
        with torch.no_grad():
            predicted_ids = model.generate(
                input_features,
                forced_decoder_ids=model.processor.get_decoder_prompt_ids(language="ko", task="transcribe")
            )
        
        # 결과 디코딩
        transcription = model.processor.batch_decode(predicted_ids, skip_special_tokens=True)[0]
        
        # 결과 출력
        print(f"[{start / sampling_rate:.2f}s - {end / sampling_rate:.2f}s]: {transcription}")

# 실행
audio_path = "data1.wav"  # 음원 파일 경로
fast_transcription(audio_path, chunk_size=2.0, sampling_rate=16000)